In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import math as m
import warnings
warnings.filterwarnings('ignore')

In [2]:
'''
If gpu is available we will use it
'''
use_cuda = True

In [3]:
class CNN(nn.Module):

    def __init__(self):

        super(CNN, self).__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(1, O_1, K_1), nn.ReLU(),
                                   nn.MaxPool2d(KP_1))

        self.conv2 = nn.Sequential(nn.Conv2d(O_1, O_2, K_2), nn.ReLU(),
                                   nn.MaxPool2d(KP_2))

        self.conv3 = nn.Sequential(nn.Conv2d(O_2, O_3, K_3), nn.ReLU(),
                                   nn.MaxPool2d(KP_3))

        self.conv4 = nn.Sequential(nn.Conv2d(O_3, O_4, K_4), nn.ReLU(),
                                   nn.MaxPool2d(KP_4))

        self.fc1 = nn.Linear(conv_linear_out, FN_1)

        self.fc2 = nn.Linear(FN_1, 10)

    def forward(self, x):
        x = x.float()
        x = F.leaky_relu(self.conv1(x))
        x = F.leaky_relu(self.conv2(x))
        x = F.leaky_relu(self.conv3(x))
        x = F.leaky_relu(self.conv4(x))
        x = x.view(len(x), -1)
        x = F.logsigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
# trained on Y00-Y09_plusnoise:
#net = torch.load('/disk1/pore_data/NanoporeTERs/trained_cnn_jeff_03132019.pt')

# trained on AGNPRT_plusnoise:
net = torch.load('/disk1/pore_data/jeff_saved/NTERs_trained_cnn_05152019.pt')

net.eval()

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 17, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(17, 18, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(18, 32, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(32, 37, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=592, out_features=148, bias=True)
  (fc2): Linear(in_features=148, out_features=10, bias=True)
)

In [5]:
'''
testing
'''

test_data_list2 = ['20190627_dSmtNP_0_3_4_raw_windows.npy']


# confidence threshold of whether or not to consider a prediction can be adjusted to
conf_thresh = 0.9

for file_name in test_data_list2:
    X_test = np.load('/disk1/pore_data/NanoporeTERs/' + str(file_name))
    file_name = file_name[:len(file_name) - 4]
    print(np.shape(X_test))
    
    # you may have to adjust the number of examples you predict per iteration, depending on memory space on gpu
    X_test = X_test[:2000, :]
    print(np.shape(X_test))
    X_test = X_test.reshape(
        len(X_test), X_test.shape[1], 1)
    X_test = X_test[:, :19881]
    X_test = X_test.reshape(len(X_test), 1, 141, 141)
    X_test = torch.from_numpy(X_test)
    X_test = X_test.cuda()
    outputs = net(X_test)
    out = nn.functional.softmax(outputs)
    prob, lab = torch.topk(out, 1)

    non_classified = 0
    for i in range(len(prob)):
        if not(prob[i] >= conf_thresh):
            non_classified += 1
            lab[i] = -1

    percent_non_classified = 100*non_classified/len(lab)
    print('Percentage not classified: ' + str(percent_non_classified))
    lab = lab.cpu().numpy()
    np.savetxt('/disk1/pore_data/NanoporeTERs/predictions_updated_20190917/' + str(file_name) +
               '_prediction_jeff20190917_confthresh_' + str(conf_thresh) + '.csv', lab, delimiter=',')
    print(str(file_name) + '_prediction_jeff20190917.csv created')

(4893, 20000)
(2000, 20000)
Percentage not classified: 89
20190627_dSmtNP_0_3_4_raw_windows_prediction_jeff20190917.csv created
